In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('DataFrame_3G_After_Cleaning.csv')

Isolation Forest

In [3]:
import plotly.express as px
from sklearn.ensemble import IsolationForest
import plotly.graph_objects as go

In [37]:
x = df['PS DROP Rate 2'].apply(lambda x: [x]).to_list()
iso_forest = IsolationForest(n_estimators=100,
                             max_samples="auto",
                             contamination=0.05,
                             )
iso_forest.fit(x)
y_pred = iso_forest.predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [38]:
df["anomaly"] = y_pred
#df.head()

In [39]:
iso_anomaly_df = pd.DataFrame(df)
iso_anomaly_df = iso_anomaly_df.loc[iso_anomaly_df['anomaly'] == 1]
#iso_anomaly_df.head()


In [40]:
fig = px.line(df,x='index', y='PS DROP Rate 2', title='Unsupervised anomaly detection in PS DROP Rate ')
fig.add_trace(go.Scatter(x=iso_anomaly_df["index"].to_list(), y=iso_anomaly_df["PS DROP Rate 2"].to_list(),
                            mode='markers', name='anomalies'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [41]:
nbr_val_inf_threshold = len(df[df['PS DROP Rate 2'] > 3.00])
print("Nombres des valeurs inférieur à 2.00% sont :", nbr_val_inf_threshold)
nbr_all_anomaly_PSDR = len(df[df['anomaly'] == 1])
nbr_true_anom_PSDR = len(df[(df['anomaly'] == 1) & (df['PS DROP Rate 2'] > 3.00)])
print("Nombres des vrais anomalie détecter sont :",nbr_true_anom_PSDR," parmi ",nbr_all_anomaly_PSDR, "anomalies détecter")

Nombres des valeurs inférieur à 2.00% sont : 14
Nombres des vrais anomalie détecter sont : 9  parmi  12 anomalies détecter


Local Outlier Factor

In [56]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=10, contamination=0.1)
y_pred = lof.fit_predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [57]:
df["anomaly"] = y_pred
#df.head()

In [58]:
lof_anomaly_df = pd.DataFrame(df)
lof_anomaly_df = lof_anomaly_df.loc[lof_anomaly_df['anomaly'] == 1]
#lof_anomaly_df.head()

In [59]:
fig = px.line(df, x='index', y='PS DROP Rate 2', title='Unsupervised anomaly detection in PS DROP Rate 2')
fig.add_trace(go.Scatter(x=lof_anomaly_df["index"].to_list(), y=lof_anomaly_df["PS DROP Rate 2"].to_list(),
                        mode='markers', name='anomalies'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [60]:
nbr_val_inf_threshold = len(df[df['PS DROP Rate 2'] > 3.00])
print("Nombres des valeurs inférieur à 2.00% sont :", nbr_val_inf_threshold)
nbr_all_anomaly_PSDR = len(df[df['anomaly'] == 1])
nbr_true_anom_PSDR = len(df[(df['anomaly'] == 1) & (df['PS DROP Rate 2'] > 3.00)])
print("Nombres des vrais anomalie détecter sont :",nbr_true_anom_PSDR," parmi ",nbr_all_anomaly_PSDR, "anomalies détecter")

Nombres des valeurs inférieur à 2.00% sont : 14
Nombres des vrais anomalie détecter sont : 6  parmi  23 anomalies détecter


OneClassSVM

In [14]:
from sklearn.svm import OneClassSVM
ocsvm = OneClassSVM(kernel='rbf',
                     gamma="auto",
                     nu=0.1)
y_pred = ocsvm.fit_predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [15]:
df["anomaly"] = y_pred
df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
0,0,16-12-21,0,74.39,66.74,7.65,156.83,99.69,0.77,95.94,1.72,100.0,99.86,93.94,100.00,755.79,0
1,1,17-12-21,0,65.41,58.40,7.01,107.82,99.26,0.97,95.40,2.37,100.0,99.67,97.98,100.00,755.79,0
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.0,99.57,96.43,0.00,755.79,0
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.0,99.72,93.14,100.00,755.79,0
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.0,99.86,94.23,66.67,755.79,1


In [16]:
ocsvm_anomaly_df = pd.DataFrame(df)
ocsvm_anomaly_df = ocsvm_anomaly_df.loc[ocsvm_anomaly_df['anomaly'] == 1]
ocsvm_anomaly_df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.00,99.86,94.23,66.67,755.79,1
10,10,26-12-21,0,61.63,54.18,7.46,92.98,99.14,3.44,95.36,3.75,97.34,99.62,94.97,50.00,639.02,1
16,16,01-01-22,0,68.80,59.75,9.05,98.97,99.00,3.71,95.40,3.64,96.73,99.51,94.93,100.00,634.37,1
17,17,01-02-22,0,72.00,63.46,8.53,104.23,99.16,3.10,95.68,3.33,96.98,99.66,96.73,100.00,645.12,1
24,24,01-09-22,0,72.46,61.93,10.53,89.79,98.41,3.48,93.11,4.73,96.95,98.69,92.60,100.00,490.41,1


In [17]:
fig = px.line(df, x='index', y='PS DROP Rate 2', title='Unsupervise anomaly detection in PS DROP Rate 2')
fig.add_trace(go.Scatter(x=ocsvm_anomaly_df["index"].to_list(),
                        y=ocsvm_anomaly_df["PS DROP Rate 2"].to_list(),
                        mode='markers',
                        name='anomaly'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [18]:
nbr_val_inf_threshold = len(df[df['PS DROP Rate 2'] > 3.00])
print("Nombres des valeurs inférieur à 2.00% sont :", nbr_val_inf_threshold)
nbr_all_anomaly_PSDR = len(df[df['anomaly'] == 1])
nbr_true_anom_PSDR = len(df[(df['anomaly'] == 1) & (df['PS DROP Rate 2'] > 3.00)])
print("Nombres des vrais anomalie détecter sont :",nbr_true_anom_PSDR," parmi ",nbr_all_anomaly_PSDR, "anomalies détecter")

Nombres des valeurs inférieur à 2.00% sont : 14
Nombres des vrais anomalie détecter sont : 7  parmi  21 anomalies détecter
